# Top 10 selector

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df = pd.read_csv('../data/10%sample/2019-Oct.csv_10%.csv')

In [22]:
df.shape

(4248118, 9)

In [3]:
df.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-10-01 00:00:17 UTC,view,23100006,2053013561638126333,NaN,NaN,357.79,513642368,17566c27-0a8f-4506-9f30-c6a2ccbf583b
1,2019-10-01 00:00:20 UTC,view,4803399,2053013554658804075,electronics.audio.headphone,jbl,33.21,555428858,8a6afed4-77f8-40c9-8e76-e062b28216ce
2,2019-10-01 00:00:23 UTC,view,6200260,2053013552293216471,appliances.environment.air_heater,midea,47.62,538645907,7d9a8784-7b6c-426e-9924-9f688812fd71
3,2019-10-01 00:00:58 UTC,view,4802639,2053013554658804075,electronics.audio.headphone,sony,218.77,514808401,1877639d-46a4-44f8-bae9-a14456952240
4,2019-10-01 00:01:11 UTC,view,1004836,2053013555631882655,electronics.smartphone,samsung,241.19,546259103,6e2984c8-502e-4fe7-bbba-34087f760175


In [21]:
# df.query("brand == 'apple' and category_code == 'sport bicycle'")  #check the dataframe output
df.query("product_id == 1002544")

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
119,2019-10-01 02:17:13 UTC,view,1002544,2053013555631882655,electronics.smartphone,apple,464.13,515544667,f619453a-5696-43ba-8cd5-3752e31d811d
163,2019-10-01 02:19:44 UTC,view,1002544,2053013555631882655,electronics.smartphone,apple,464.13,554571432,8aeea188-8933-4756-a96b-f9eb6893f1a8
283,2019-10-01 02:23:15 UTC,view,1002544,2053013555631882655,electronics.smartphone,apple,464.13,479233261,44dd8d4a-5fa0-44a8-9729-4720cb8cc98f
434,2019-10-01 02:26:42 UTC,view,1002544,2053013555631882655,electronics.smartphone,apple,464.13,546213844,42ae575d-3316-4b2d-98cd-ae60756542c3
503,2019-10-01 02:28:10 UTC,view,1002544,2053013555631882655,electronics.smartphone,apple,464.13,519830620,a03d035b-3275-4db1-8467-b4650f249145
...,...,...,...,...,...,...,...,...,...
4246143,2019-10-31 21:34:33 UTC,view,1002544,2053013555631882655,electronics.smartphone,apple,458.28,512640398,e11d7cfe-fc41-4659-8223-6d7f6a244bbf
4246498,2019-10-31 21:58:25 UTC,view,1002544,2053013555631882655,electronics.smartphone,apple,458.28,563557069,b44541e9-ab55-4272-a52c-16d0d58bdda6
4246737,2019-10-31 22:14:51 UTC,view,1002544,2053013555631882655,electronics.smartphone,apple,458.28,550264978,2b130498-46c6-42df-bfc3-2ec802e91280
4246776,2019-10-31 22:17:48 UTC,view,1002544,2053013555631882655,electronics.smartphone,apple,458.28,522614060,bb9b2f2c-ff9f-4c31-bfee-c0765a682a18


In [5]:
def preprocessing_feat(X):    
    X_preprocessed = X.dropna(subset = ['category_code', 'brand']) #tbd!!
    X_preprocessed = X_preprocessed.drop_duplicates()
    X_preprocessed['category_code'] = X_preprocessed['category_code'].str.replace('.',' ')
    return X_preprocessed

def pricing_criterion(X):
    pricing_guide = X.groupby('category_code')['price'].describe()[["25%", "75%"]].reset_index()
    X_merged = X.merge(pricing_guide, on="category_code", how="right")
    X_merged["price_category"] = X_merged.apply(lambda row: make_column(row), axis=1)
    return X_merged

def make_column(row):
    if row["price"] < row["25%"]:
        return "low"
    elif row["price"] < row["75%"]:
        return "medium"
    else:
        return "high"

def metadata(X):
    X['metadata'] = X[['category_code', 'brand', 'price_category']].apply(lambda x: ' '.join(x), axis = 1)
    return X

def rating(X):
    dct = {'view': 1, 'cart': 3, 'purchase': 5}
    X['rating'] = X['event_type'].map(dct)
    return X

In [6]:
X_prep = preprocessing_feat(df)

/var/folders/6f/vn_j9pqn1qd8rp92y6x0h0hh0000gn/T/ipykernel_23140/3456390047.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  X_preprocessed['category_code'] = X_preprocessed['category_code'].str.replace('.',' ')


In [7]:
# X_prep.query("brand == 'apple' and category_code == 'sport bicycle'")  #check the dataframe output
X_prep.query("product_id == 4804718")

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session


In [8]:
X_pric = pricing_criterion(X_prep)

In [9]:
X_rating = rating(X_pric)

In [10]:
def metadata(X):
    X['metadata'] = X[['category_code', 'brand', 'price_category']].apply(lambda x: ' '.join(x), axis = 1)
    return X

In [11]:
X_meta = metadata(X_pric)

In [12]:
X_meta.query("brand == 'apple' and category_code == 'sport bicycle'")
# df_m.query("user_session == 'bde905b5-20f1-40ff-8ef6-a60d875e31e0'")

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session,25%,75%,price_category,rating,metadata


In [13]:
def ranking(X, n=5):
    X = pd.DataFrame(X_rating.groupby(by='product_id').rating.sum()).sort_values('rating', ascending=False).reset_index().index[0:n]
    return X
    

In [14]:
# def top_n(product_id, X_rating, n=10):
    
#     """select top n products by ranking'"""
    
#     rating_idx = pd.DataFrame(X_rating.groupby(by='product_id').rating.sum()).sort_values('rating', ascending=False).reset_index().index[0:n]
#     return rating_idx

#     counter = 0
    
#     for i in rating_idx:
#         meta_text = X_rating[X_rating['product_id'] == i][['metadata']].iloc[0,:][0]
        
#         if counter == 0:
#             print(f"Top {n} recommendations for product_id {product_id}:")
#             print(f"{i} - {meta_text} \n")
# #             print("-----------------------")

#         else:
#             print(f"Rec {counter}) {i} - {meta_text}")
        
#         counter += 1

In [15]:
def top_n_overall(product_id, X_rating, n=5):
    
    """select top n products by ranking'"""
    
    rating_idx = pd.DataFrame(X_rating.groupby(by='product_id').rating.sum()).sort_values('rating', ascending=False).index[0:n]
    #     rec_df.sort_values(ranking, ascending=False).index[0:n]
    counter = 0
    
    for i in rating_idx:
        meta_text = X_meta[X_meta['product_id'] == i][['metadata']].iloc[0,:][0]
        
        if counter == 0:
            print(f"Top {n} recommendations for product_id {product_id}:")
            print(f"{i} - {meta_text} \n")
#             print("-----------------------")

        else:
            print(f"Rec {counter}) {i} - {meta_text}")
        
        counter += 1

In [16]:
top_n_overall(49800017, X_rating)

Top 5 recommendations for product_id 49800017:
1004856 - electronics smartphone samsung low 

Rec 1) 1004767 - electronics smartphone samsung medium
Rec 2) 1005115 - electronics smartphone apple high
Rec 3) 1004833 - electronics smartphone samsung low
Rec 4) 4804056 - electronics audio headphone apple high


In [17]:
def top_n_brands(X_rating, n=20):
    
    """select top n products by ranking'"""
    
    rating_idx =     pd.DataFrame(X_rating.groupby(['brand']).rating.sum()).sort_values('rating', ascending=False).index[0:n]

    #     rec_df.sort_values(ranking, ascending=False).index[0:n]
    return list(rating_idx)

In [18]:
top_n_brands(X_rating)

['samsung',
 'apple',
 'xiaomi',
 'huawei',
 'oppo',
 'lg',
 'acer',
 'lenovo',
 'bosch',
 'indesit',
 'hp',
 'artel',
 'respect',
 'casio',
 'elenberg',
 'sony',
 'beko',
 'asus',
 'haier',
 'nokia']

In [19]:
def top_n_by_brand(X_rating, brand, n=10):
    
    """select top n products by brand'"""
    
    rating_idx = pd.DataFrame(X_rating.groupby(['product_id','brand']).rating.sum()).sort_values('rating', ascending=False).reset_index('brand')
    rating_idx = rating_idx[rating_idx['brand']==brand].index[0:n]
#     return rating_idx

    counter = 0

    for i in rating_idx:
            meta_text = X_meta[X_meta['product_id'] == i][['metadata']].iloc[0,:][0]

            if counter == 0:
                print(f"Top {n} recommendations for product_id {brand}:")
                print(f"{i} - {meta_text} \n")
    #             print("-----------------------")

            else:
                print(f"Rec {counter}) {i} - {meta_text}")

            counter += 1

In [20]:
top_n_by_brand(X_rating, brand='apple')

Top 10 recommendations for product_id apple:
1005115 - electronics smartphone apple high 

Rec 1) 4804056 - electronics audio headphone apple high
Rec 2) 1004249 - electronics smartphone apple high
Rec 3) 1002544 - electronics smartphone apple medium
Rec 4) 1005105 - electronics smartphone apple high
Rec 5) 1002524 - electronics smartphone apple medium
Rec 6) 1002633 - electronics smartphone apple medium
Rec 7) 1003306 - electronics smartphone apple medium
Rec 8) 1005135 - electronics smartphone apple high
Rec 9) 1003317 - electronics smartphone apple high
